In [1]:
# from tkinter import *
# from tkinter.ttk import Combobox
# from PIL import ImageTk
# from bs4 import BeautifulSoup
# import requests
# import re
# import os
#WORKS!!!!!!
import threading
from tkinter import *
from tkinter.ttk import Combobox
from PIL import Image, ImageTk
import requests
from bs4 import BeautifulSoup
import re


In [ ]:
class CricketScore:
    def __init__(self, rootWindow):
        self.rootWindow = rootWindow
        self.rootWindow.title('LIVE CRICKET SCORE')
        self.rootWindow.geometry('800x500+100+100')  # Set position (x=100, y=100)

        # Background Image Load
        try:
            self.bg_image = ImageTk.PhotoImage(Image.open("cricketpng.jpg").resize((800, 500)))
            self.bg_label = Label(self.rootWindow, image=self.bg_image)
            self.bg_label.place(x=0, y=0)
        except Exception as e:
            print("Error loading image:", e)

        # Label for title
        self.label = Label(self.rootWindow, text='Live Matches', font=("times new roman", 30), bg="lightblue", fg="black")
        self.label.place(x=250, y=50)

        # Combobox for selecting match
        self.var = StringVar()
        self.matches = self.match_details()
        self.data = list(self.matches.keys())
        self.cb = Combobox(self.rootWindow, values=self.data, textvariable=self.var, width=50, state="readonly")
        self.cb.place(x=250, y=200)
        self.cb.set("Select a Match")

        # Button to check score
        self.b1 = Button(self.rootWindow, text="Check Score", font=("times new roman", 15), command=self.show_match_details)
        self.b1.place(x=350, y=250)

    def show_match_details(self):
        selected_match = self.var.get()
        if selected_match not in self.matches:
            print("Invalid match selection")
            return

        # Clear previous frame if exists
        for widget in self.rootWindow.winfo_children():
            if isinstance(widget, Frame):
                widget.destroy()

        self.frame1 = Frame(self.rootWindow, bg="#ADD8E6")
        self.frame1.place(x=100, y=300, width=600, height=150)

        match_data = self.matches[selected_match]

        Label(self.frame1, text=f"{selected_match} - {match_data['match_header']}", font=("times new roman", 15, "bold"), bg="#ADD8E6").place(x=10, y=10)
        Label(self.frame1, text=f"Score Details: {match_data['score_card']}", font=("times new roman", 12), bg="#ADD8E6").place(x=10, y=50)
        Label(self.frame1, text=f"Summary: {match_data['summary']}", font=("times new roman", 12), bg="#ADD8E6").place(x=10, y=90)

    def match_details(self):
        details = self.scrap()
        live_match = {}
        for detail in details:
            live_team_details = {}
            summary = self.match_header(detail)
            if summary:
                match_header = self.match_header(detail)
                teams = self.teams_name(detail)
                score_card = self.team_score(detail)
                live_team_details['summary'] = summary
                live_team_details['match_header'] = match_header
                live_team_details['score_card'] = score_card[0] + " :: " + score_card[1]
                live_match[f"{teams[0]} vs {teams[1]}"] = live_team_details
        return live_match

    def team_score(self, detail):
        t = []
        team1_details = self.safe_find(detail, "div", class_="cb-hmscg-bat-txt")
        team2_details = self.safe_find(detail, "div", class_="cb-hmscg-bwl-txt")
        t.append(team1_details)
        t.append(team2_details)
        return t

    def teams_name(self, detail):
        t = []
        team1_details = self.safe_find(detail, "div", class_="cb-hmscg-bat-txt")
        team2_details = self.safe_find(detail, "div", class_="cb-hmscg-bwl-txt")

        # Extracting team names before scores
        team1_index = re.search(r"\d", team1_details).start() if re.search(r"\d", team1_details) else len(team1_details)
        team2_index = re.search(r"\d", team2_details).start() if re.search(r"\d", team2_details) else len(team2_details)
        t.append(team1_details[:team1_index])
        t.append(team2_details[:team2_index])
        return t

    def match_header(self, detail):
        match_header = detail.find("div", class_="cb-mtch-crd-hdr")
        return match_header.text.strip() if match_header else "No Match Header"

    def safe_find(self, detail, tag, class_):
        element = detail.find(tag, class_=class_)
        return element.text.strip() if element else "Data not available"

    def scrap(self):
        try:
            URL = "https://www.cricbuzz.com/"
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, "html.parser")
            results = soup.find(id="match_menu_container")
            return results.find_all("li", class_="cb-match-card")
        except Exception as e:
            print("Error scraping data:", e)
            return []


def main():
    rootWindow = Tk()
    CricketScore(rootWindow)
    rootWindow.mainloop()


if __name__ == '__main__':
    main()

Error loading image: [Errno 2] No such file or directory: 'C:\\Users\\Syeda Mehak Zahra\\Desktop\\dcs open\\cricketpng'
